In [1]:
import numpy as np
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import datetime
import matplotlib.pyplot as plt

In [2]:
dataF=pd.read_csv('C:/Users/saragada/Desktop/HPI/Forecast/bw/bw.data', sep='\034',encoding='latin-1',error_bad_lines=False)

C:\Users\saragada\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning:

Columns (0,2,10,26,39,40,41,44,45,46,47,49,50,51,52,53,54,55,56,57,58,59) have mixed types. Specify dtype option on import or set low_memory=False.



In [4]:
dataF=dataF[pd.notnull(dataF['CLR_DT'])]

In [9]:
from datetime import datetime
from dateutil import relativedelta

months=list()
#This will find the difference between the two dates
for i in range(0,len(dataF)):
    difference = relativedelta.relativedelta(datetime.strptime(dataF['VAL_DT'].iloc[i],'%Y-%m-%d'),datetime.strptime(dataF['POST_DT'].iloc[i],'%Y-%m-%d'))
    months.append(difference.months)

In [10]:
dataF['Category']=months

In [28]:
a=dataF['Category'].unique()

In [29]:
for i in a:
    if int(i)<=4:
       
        datak=dataF[dataF['Category']==i]
        datak.to_csv('C:/Users/saragada/Desktop/HPI/Fresh Start Forecast/Data/dataslice'+str(i)+'.csv')
    else:
        datak=dataF[dataF['Category']>4]
        datak.to_csv('C:/Users/saragada/Desktop/HPI/Fresh Start Forecast/Data/dataslice_rest.csv')
        
        

In [102]:
import os
arr = os.listdir('C:/Users/saragada/Desktop/HPI/Fresh Start Forecast/Data/')


In [103]:
arr

['dataslice0.csv',
 'dataslice1.csv',
 'dataslice2.csv',
 'dataslice3.csv',
 'dataslice4.csv',
 'dataslice_rest.csv']

In [104]:
import pandas as pd
import numpy as np
from HPprophecy import Prophecy
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
 
%matplotlib inline
 
plt.rcParams['figure.figsize']=(30,10)
plt.style.use('ggplot')

In [132]:
def foresee(arr):
    log_file=pd.DataFrame()
    log_file2=pd.DataFrame()
    
    filelog=list()
    for file in arr:
        data1=pd.read_csv('C:/Users/saragada/Desktop/HPI/Fresh Start Forecast/Data/'+str(file),encoding='latin-1')
        df1=data1.groupby('VAL_DT', as_index=False).agg({"USD_AMT": "sum"})
        df1=df1.reset_index()
        df1=df1[['VAL_DT','USD_AMT']]
        df1.rename(columns={'VAL_DT': 'ds', 'USD_AMT': 'y'}, inplace=True)
        df1=df1[(df1['ds']>'2016-08-01')]
        df=df1[(df1['ds']<'2018-01-01')]
        df['ds'] =pd.to_datetime(df.ds)
        df1['ds'] =pd.to_datetime(df1.ds)
        df.index = df['ds']
        df1.index = df1['ds']
        df=df.resample('M').sum()
        df=df.reset_index()
        df1=df1.resample('M').sum()
        df1=df1.reset_index()
        tuner=pd.DataFrame()
        a=list()
        d=list()
        e=list()
        for scale in range(10,300,10):
            m = Prophecy()
            m.add_seasonality(name='monthly', period=12, fourier_order=8,prior_scale=scale).fit(df)

            future = m.make_future_dataframe(periods=3, freq='M')
            forecast = m.predict(future)
            a.append(scale)
            b=mean_absolute_error(forecast['yhat'][:len(df)],df['y'])
            c=mean_absolute_error(forecast['yhat'][len(df):],df1['y'][len(df):-1])
            d.append(b+c)
            filelog.append(file)
            
        tuner['Scale']=a
        tuner['error']=d
        log_file2=pd.concat([tuner,log_file2],ignore_index=True)
    log_file=log_file2
    log_file['file']=filelog
    return log_file
            

In [133]:
log_file=foresee(arr)

C:\Users\saragada\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning:

Columns (1,27,40,41,42,46,47,48,50,51,52,53,54,56,57,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Users\saragada\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.fore

INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disab

INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:

INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disab

INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:

INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disab

INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HP

INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 12.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:

INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 11.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 11.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:

INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 11.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 11.0.
INFO:HPprophecy.forecaster:Disabling yearly seasonality. Run prophecy with yearly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling weekly seasonality. Run prophecy with weekly_seasonality=True to override this.
INFO:HPprophecy.forecaster:Disabling daily seasonality. Run prophecy with daily_seasonality=True to override this.
INFO:HPprophecy.forecaster:n_changepoints greater than number of observations.Using 11.0.
INFO:HPprophecy.forecaster:Disab

In [134]:
log_file.to_csv('log_file.csv')

In [ ]:
log_file2